#Импорты

In [ ]:
!pip install pymorphy2
!pip install -U 'scikit-learn<0.24'
!pip install stanza
!pip install sklearn-crfsuite

In [2]:
import pandas as pd
from tqdm.autonotebook import tqdm
from sklearn.model_selection import train_test_split
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from collections import defaultdict
import stanza
import pickle
from pymorphy2 import MorphAnalyzer
stanza.download('ru')
morph = MorphAnalyzer()

<ipython-input-2-9b0a19d70465>:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


INFO:stanza:Downloading default packages for language: ru (Russian) ...


INFO:stanza:Finished downloading models and saved to /root/stanza_resources.


#1 Выделение аспектов

##Загрузка трейна и приведение данных в формат IOB

In [3]:
!wget https://github.com/named-entity/hse-nlp/raw/master/4th_year/Project/train_split_aspects.txt
!wget https://github.com/named-entity/hse-nlp/raw/master/4th_year/Project/train_split_reviews.txt

--2023-02-09 12:43:00--  https://github.com/named-entity/hse-nlp/raw/master/4th_year/Project/train_split_aspects.txt
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/train_split_aspects.txt [following]
--2023-02-09 12:43:00--  https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/train_split_aspects.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 170883 (167K) [text/plain]
Saving to: ‘train_split_aspects.txt’

train_split_aspects 100%[===================>] 166.88K  --.-KB/s    in 0.002s  

2023-02-09 12:43:00 (68.2 MB/s) - 

In [4]:
reviews = {}
with open('train_split_reviews.txt') as f:
  for line in f:
    line = line.rstrip('\r\n').split('\t')
    reviews[line[0]] = line[1]
     

aspects = defaultdict(list)
with open('train_split_aspects.txt') as f:
  for line in f:
    line = line.rstrip('\r\n').split('\t')
    keys = ('category', 'mention', 'start', 'end', 'sentiment')
    aspects[line[0]].append(dict(zip(keys, line[1:])))

In [5]:
nlp = stanza.Pipeline('ru', processors='tokenize')

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Done loading processors!


In [6]:
def get_IOB(texts):
  iob = []
  with tqdm(total=len(texts)) as pbar:
    for text_id, text in texts.items():
      iob_text = []
      processed = nlp(text)
      for token in processed.iter_tokens():
        add = False
        parsed = morph.parse(token.text)[0]
        postag = parsed.tag.POS
        if postag is None:
          if token.text.isnumeric():
            postag = 'NUMB'
          else:
            postag = 'PUNKT'
        for mention in aspects[text_id]:
          if token.start_char == int(mention['start']) and token.end_char <= int(mention['end']):
            iob_text.append({'text':token.text, 'pos': postag, 'tag':'B-'+mention['category'],
                          'text_id': text_id, 'start': token.start_char, 'end': token.end_char})
            add = True
          elif token.start_char > int(mention['start']) and token.end_char <= int(mention['end']):
            iob_text.append({'text':token.text, 'pos': postag, 'tag':'I-'+mention['category'],
                          'text_id': text_id, 'start': token.start_char, 'end': token.end_char})
            add = True
        if not add:
          iob_text.append({'text':token.text, 'pos': postag, 'tag':'O', 
                        'text_id': text_id, 'start': token.start_char, 
                        'end': token.end_char})
      iob.append(iob_text)
      pbar.update(1)

  return iob
        

In [7]:
IOB = get_IOB(reviews)

  0%|          | 0/213 [00:00<?, ?it/s]

##CRF. Обучение модели

In [8]:
!wget https://github.com/named-entity/hse-nlp/raw/master/4th_year/Project/dev_reviews.txt
!wget https://github.com/named-entity/hse-nlp/raw/master/4th_year/Project/dev_aspects.txt

--2023-02-09 12:44:18--  https://github.com/named-entity/hse-nlp/raw/master/4th_year/Project/dev_reviews.txt
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/dev_reviews.txt [following]
--2023-02-09 12:44:19--  https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/dev_reviews.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 110515 (108K) [text/plain]
Saving to: ‘dev_reviews.txt’

dev_reviews.txt     100%[===================>] 107.92K  --.-KB/s    in 0.002s  

2023-02-09 12:44:19 (42.9 MB/s) - ‘dev_reviews.txt’ saved [110515/

In [9]:
dev_reviews = {}
with open('dev_reviews.txt') as f:
  for line in f:
    line = line.rstrip('\r\n').split('\t')
    dev_reviews[line[0]] = line[1]
     

dev_aspects = defaultdict(list)
with open('dev_aspects.txt') as f:
  for line in f:
    line = line.rstrip('\r\n').split('\t')
    keys = ('category', 'mention', 'start', 'end', 'sentiment')
    dev_aspects[line[0]].append(dict(zip(keys, line[1:])))

In [10]:
def word2features(sent, i):
    word = sent[i]['text']
    postag = sent[i]['pos']
    text_id = sent[i]['text_id']
    start = sent[i]['start']
    end = sent[i]['end']

    features = {
        'text_id': text_id, 
        'word': word,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'start': start,
        'end': end
    }
    if i > 0:
        word1 = sent[i-1]['text']
        postag1 = sent[i-1]['pos']
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
        })
    else:
        features['BOS'] = True
        
    if i < len(sent)-1:
        word1 = sent[i+1]['text']
        postag1 = sent[i+1]['pos']
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True
                
    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [word['tag'] for word in sent]

In [11]:
def text_to_dict(texts):
  rev_dicts = []
  for text_id, text in texts.items():
    tokens = []
    processed = nlp(text)
    for token in processed.iter_tokens():
      parsed = morph.parse(token.text)[0]
      postag = parsed.tag.POS
      if postag is None:
        if token.text.isnumeric():
          postag = 'NUMB'
        else:
          postag = 'PUNKT'
      tokens.append({'text':token.text, 'pos':postag, 'text_id':text_id,
                     'start':token.start_char, 'end':token.end_char})
    rev_dicts.append(tokens)
      
  return rev_dicts

In [12]:
def get_predicted(y_predicted, X_test):
  data = {'text_id': [], 'tag': [], 'text': [], 'start': [], 'end': []}
  for i, tags in enumerate(y_pred):
    for j, tag in enumerate(tags):
      if tag!='O':
        if tags[j].startswith('B'):
          if len(tags) > j+1:
            if tags[j+1].startswith('I'):
              tokens = []
              tokens.append(X_test[i][j]['word'])
              start = X_test[i][j]['start']
              k = j + 1
              while tags[k].startswith('I'):
                tokens.append(X_test[i][k]['word'])
                k += 1
              end = X_test[i][k-1]['end']

              data['text_id'].append(X_test[i][j]['text_id'])
              data['tag'].append(tag.split('-')[1])
              data['text'].append(' '.join(tokens))
              data['start'].append(start)
              data['end'].append(end)
            elif tags[j].startswith('B') and not tags[j+1].startswith('I'):
              start = X_test[i][j]['start']
              end = X_test[i][j]['end']

              data['text_id'].append(X_test[i][j]['text_id'])
              data['tag'].append(tag.split('-')[1])
              data['text'].append(X_test[i][j]['word'])
              data['start'].append(start)
              data['end'].append(end)
          else:
            start = X_test[i][j]['start']
            end = X_test[i][j]['end']

            data['text_id'].append(X_test[i][j]['text_id'])
            data['tag'].append(tag.split('-')[1])
            data['text'].append(X_test[i][j]['word'])
            data['start'].append(start)
            data['end'].append(end)


  df = pd.DataFrame(data)
  return df

In [13]:
X_test = [sent2features(s) for s in text_to_dict(dev_reviews)]

X_train = [sent2features(s) for s in IOB]
y_train = [sent2labels(s) for s in IOB]

In [14]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    c1=0.1, 
    c2=0.1, 
    max_iterations=100, 
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [15]:
y_pred = crf.predict(X_test)

In [339]:
pred = get_predicted(y_pred, X_test)
with open('dev_pred_asp_no_sent.txt', 'w') as f:
  for text_id, tag, text, start, end in zip(pred['text_id'], pred['tag'], pred['text'], pred['start'], pred['end']):
      print(text_id, tag, text, start, end, sep="\t", file=f)

In [17]:
filename = 'crf_model.sav'
pickle.dump(crf, open(filename, 'wb'))

#2 Оценка

In [ ]:
!pip install dostoevsky
!python -m dostoevsky download fasttext-social-network-model
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel

In [73]:
print(aspects)
#reviews
dev_aspects
dev_reviews

defaultdict(<class 'list'>, {'30808': [{'category': 'Whole', 'mention': 'ресторане', 'start': '16', 'end': '25', 'sentiment': 'neutral'}, {'category': 'Interior', 'mention': 'первом этаже', 'start': '43', 'end': '55', 'sentiment': 'neutral'}, {'category': 'Whole', 'mention': 'руководству ресторана', 'start': '124', 'end': '145', 'sentiment': 'positive'}, {'category': 'Service', 'mention': 'обслуживающему персоналу', 'start': '147', 'end': '171', 'sentiment': 'positive'}, {'category': 'Service', 'mention': 'сотрудникам', 'start': '189', 'end': '200', 'sentiment': 'positive'}, {'category': 'Food', 'mention': 'блюда', 'start': '254', 'end': '259', 'sentiment': 'positive'}, {'category': 'Interior', 'mention': 'интерьер', 'start': '310', 'end': '318', 'sentiment': 'positive'}, {'category': 'Food', 'mention': 'Стол', 'start': '337', 'end': '341', 'sentiment': 'positive'}, {'category': 'Service', 'mention': 'обслуживал', 'start': '360', 'end': '370', 'sentiment': 'positive'}, {'category': 'Se

In [222]:
def get_chunk(reviews, aspects, chunk_length=6):
  chunks= []
  ids = []
  for i, text in enumerate(reviews.items()):
    processed = nlp(text[1])
    tokens = []
    for t in processed.iter_tokens():
      tokens.append(t.text)
    for mention in aspects[text[0]]:
      chunk=[]
      start_chunk = []
      end_chunk = []
      start = mention['start']
      end = mention['end']
      for j, token in enumerate(processed.iter_tokens()):
        if int(token.start_char) == int(start):
          if j - chunk_length >= 0:
            start_chunk = [tokens[j-item] for item in range(chunk_length, 0, -1)]
            start_chunk.append(mention['mention'])
            chunk = start_chunk 
          else:
            start_chunk = [tokens[j-item] for item in range(j, 0, -1)]
            start_chunk.append(mention['mention'])
            chunk = start_chunk
        if int(token.end_char) == int(end):
          if j + chunk_length <= len(tokens)-1:
            end_chunk = [tokens[j+item] for item in range(1, chunk_length+1)]
            chunk.extend(end_chunk)
          else:
            end_chunk = [tokens[j+item] for item in range(1, len(tokens)-j)]
            chunk.extend(end_chunk)
      chunks.append(' '.join(chunk))
      ids.append(text[0])
  return ids, chunks

In [340]:
predicted = defaultdict(list)
with open('dev_pred_asp_no_sent.txt') as f:
  for line in f:
    line = line.rstrip('\r\n').split('\t')
    keys = ('category', 'mention', 'start', 'end')
    predicted[line[0]].append(dict(zip(keys, line[1:])))

In [245]:
tokenizer = RegexTokenizer()
model = FastTextSocialNetworkModel(tokenizer=tokenizer)

ids, messages = get_chunk(dev_reviews, predicted)

results = model.predict(messages, k = 5)

sents = []
for ids, sentiment in zip(ids, results):
    del sentiment['skip']
    del sentiment['speech']
    #print(ids, max(sentiment, key=sentiment.get))
    sents.append(max(sentiment, key=sentiment.get))

In [246]:
len(sents)

972

In [247]:
pred['sentiment'] = sents

In [249]:
pred.head()

,text_id,tag,text,start,end,sentiment
0,13823,Whole,""" аппетит """,7,16,negative
1,13823,Service,встретил,138,146,neutral
2,13823,Service,менеджер,147,155,positive
3,13823,Service,девушка,179,186,negative
4,13823,Service,проводила к столу,188,205,negative


In [252]:
with open('dev_pred_aspects.txt', 'w') as f:
  for text_id, tag, text, start, end, sentiment in zip(pred['text_id'], pred['tag'], pred['text'], pred['start'], pred['end'], pred['sentiment']):
      print(text_id, tag, text, start, end, sentiment, sep="\t", file=f)

#3 Общая оценка

In [270]:
pred.loc[pred['sentiment'] == 'negative', 'sentiment'] = -1
pred.loc[pred['sentiment'] == 'neutral', 'sentiment'] = 0
pred.loc[pred['sentiment'] == 'positive', 'sentiment'] = 1

In [333]:
def cats_total(pred):
  cats_total = []
  text_ids = pred['text_id'].unique()
  total_ids = []
  for i in text_ids:
    cats = {'Whole':0, 'Service':0, 'Food':0, 'Interior':0, 'Price':0}
    for key in cats.keys():
      total_ids.append(i)
      if len(pred.loc[(pred['text_id'] == i)&(pred['tag'] == key)]) > 0:
        upd_sum = pred.loc[(pred['text_id'] == i)&(pred['tag'] == key)]['sentiment'].sum()
        cats[key] = upd_sum
      else:
        cats[key] = 'absence'
    for k, v in cats.items():
      if type(v) != str:
        if v > 0:
          cats[k] = 'positive'
        elif v == 0:
          cats[k] = 'neutral'
        elif v <0:
          cats[k] = 'negative'
    cats_total.append(cats)
  return total_ids, cats_total

In [334]:
text_ids, pred_cats = cats_total(pred)
df = pd.DataFrame([(i, j) for a in pred_cats for i, j in a.items()],columns=['category', 'sentiment'])
df.insert(loc=0, column='text_id', value=text_ids)

In [337]:
df.to_csv('dev_pred_cats', sep='\t', header=False, index=False)